In [6]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

In [7]:
import psycopg2

conn = psycopg2.connect(host=get_secret('postgresql')['host'],
                      user=get_secret('postgresql')['username'],
                      password = get_secret('postgresql')['password'],
                      dbname=get_secret('postgresql')['engine'])

cur = conn.cursor()

In [8]:
q1_sql = """
SELECT
    year,
    pop,
    LAG(pop) OVER (ORDER BY year) AS prev_pop,
    ROUND(
        (pop - LAG(pop) OVER (ORDER BY year))::numeric
        / NULLIF(LAG(pop) OVER (ORDER BY year), 0) * 100,
        2
    ) AS growth_rate_percent
FROM population
WHERE fips = '10'
ORDER BY year ASC;
"""



In [9]:
q2_sql = """
SELECT
    p.year,
    p.pop,
    i.income
FROM population p
JOIN income i
    ON p.fips = i.fips
    AND p.year = i.year
WHERE p.fips = '10'
ORDER BY p.year ASC;
"""


In [10]:
q3_sql = """
WITH pop_diff AS (
    SELECT
        year,
        pop,
        pop - LAG(pop) OVER (ORDER BY year) AS diff
    FROM population
    WHERE fips = '10'
)
SELECT year, pop, diff
FROM pop_diff
WHERE diff IS NOT NULL
ORDER BY diff DESC
LIMIT 1;
"""
